## Let's use AutoGrad

In [1]:
import torch
import numpy
torch.set_printoptions(edgeitems=2)

In [8]:
t_c = torch.tensor([0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]) # values in unknown units


In [9]:
# applying autograd
def model(t_u, w, b):
    return w * t_u + b

In [10]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [11]:
params = torch.tensor([1.0, 0.0], requires_grad=True)

In [12]:
params.grad is None

True

In [13]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

In [14]:
params.grad

tensor([4517.2969,   82.6000])

We need to explicitly zero the gradient after using it for parameter updates.

In [17]:
# zero the gradient
if params.grad is not None:
    params.grad.zero_()

In [18]:
params.grad

tensor([0., 0.])

In [26]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()

        # Forward pass ------------------------------------------------        
        # prediction: our model, like a NN, will predict some values
        # based on some inputs, our Temperature unknown
        t_p = model(t_u, *params)

        # We'll compute some loss, like MSE (mean squared error)
        # based on the predicted values and the correct ones
        loss = loss_fn(t_p, t_c)

        # Backward pass ------------------------------------------------ 
        # all derivatives will be calculated until the leafs
        # and will accumulate those grads on the params with required_grad = True
        loss.backward()

        # step that occur on the optimizers
        with torch.no_grad():
            params -= learning_rate * params.grad
            if epoch % 500 == 0:
                print(params)

        if epoch % 500 == 0:
            print('Epoch {}, Loss {}'.format(epoch, float(loss)))
    
    return params

In [27]:
t_un = t_u * 0.1

In [28]:
training_loop(
    n_epochs=5000,
    learning_rate=1e-2,
    params=torch.tensor([1.0, 0.0], requires_grad=True),
    t_u=t_un,
    t_c=t_c
)

tensor([ 4.0443, -9.8133], requires_grad=True)
Epoch 500, Loss 7.860115051269531
tensor([  4.8021, -14.1031], requires_grad=True)
Epoch 1000, Loss 3.828537940979004
tensor([  5.1260, -15.9365], requires_grad=True)
Epoch 1500, Loss 3.092191219329834
tensor([  5.2644, -16.7200], requires_grad=True)
Epoch 2000, Loss 2.957697868347168
tensor([  5.3236, -17.0549], requires_grad=True)
Epoch 2500, Loss 2.933133840560913
tensor([  5.3489, -17.1980], requires_grad=True)
Epoch 3000, Loss 2.9286484718322754
tensor([  5.3597, -17.2591], requires_grad=True)
Epoch 3500, Loss 2.9278297424316406
tensor([  5.3643, -17.2853], requires_grad=True)
Epoch 4000, Loss 2.9276793003082275
tensor([  5.3662, -17.2964], requires_grad=True)
Epoch 4500, Loss 2.927651882171631
tensor([  5.3671, -17.3012], requires_grad=True)
Epoch 5000, Loss 2.9276468753814697


tensor([  5.3671, -17.3012], requires_grad=True)